# Step 1: Setting Up the Python Application

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, NonNegativeInt
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.chains.question_answering import load_qa_chain

In [21]:
import os

os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [26]:
# Initialise LLM

llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    max_tokens=2000
    # api_key="...",
    # base_url="...",
    # organization="...",
    # other params...
)
llm

OpenAI(client=<class 'openai.api_resources.completion.Completion'>, model_name='gpt-3.5-turbo-instruct', temperature=0.0, max_tokens=2000, openai_api_key='voc-589459551126677348884666dc80e0ebfee7.97941068', openai_api_base='https://openai.vocareum.com/v1', openai_organization='', openai_proxy='')

# Step 2: Generating Real Estate Listings

#### Note: This prompt is mentioned for information on how I generated it using ChatGPT app. From there, the listings were saved in a csv file and used ahead in the project work.

In [ ]:
listing_prompt = """
Generate 12-15 real estate listings for home, which can help a home buyer to understand various parameters, home description,  locality description, etc..  Listing should be diverse and have diverse descriptions and neighborhoods. 
Make the descriptions in listings long and more descriptive as provided in the sample listing. Ensure that they remain diverse.

A sample listing is given below:

Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""

out_listings = llm.invoke(listing_prompt)
print(out_listings)

# Step 3: Storing Listings in a Vector Database

## Vector Database Setup: 
Initialize and configure ChromaDB or a similar vector database to store real estate listings.
Generating and Storing Embeddings: Convert the LLM-generated listings into suitable embeddings that capture the semantic content of each listing, and store these embeddings in the vector database.

### Load data

In [ ]:


csvLoader = CSVLoader(file_path="./final_real_estate_listing.csv"
                     )
listings = csvLoader.load()
listings

# Scope of improvements:
# 1. Create list of documents with custom metadata colums, like price, bedrooms, etc. can go in there. 
# 2. OR Use latest version for putting numeric columns in metadata_columns
# 3. Can use a LLM to summarise a row in more verbose description matching a human description of the house and use that as page_content for Document

### Initialise embeddings

In [22]:
embed = OpenAIEmbeddings()
embed

OpenAIEmbeddings(client=<class 'openai.api_resources.embedding.Embedding'>, model='text-embedding-ada-002', deployment='text-embedding-ada-002', openai_api_version='', openai_api_base='https://openai.vocareum.com/v1', openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key='voc-589459551126677348884666dc80e0ebfee7.97941068', openai_organization='', allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=6, request_timeout=None, headers=None, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False)

### Create Vector store from the data

In [6]:
vectorStore = Chroma.from_documents(listings, embed)
vectorStore

In [8]:
# Sample query semantic similarity search

sample_query = """
Houses which are 3 bedrooms, 2 baths and are in the neighbourhood with lots of trees and away from the ocean. 
"""

# find top 3 semantically similar documents to the query
result = vectorStore.similarity_search(query=sample_query, k=3)
result

[Document(page_content="Neighborhood: Cedar Grove\nPrice: $720,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 2,100 sqft\nDescription: This beautifully updated single-story home in Cedar Grove blends modern comfort with natural serenity. Boasting 3 bedrooms and 2 full bathrooms, the home welcomes you with its airy open layout, vaulted ceilings, and large windows that frame picturesque views of the surrounding trees. The kitchen is designed for culinary enthusiasts, featuring quartz countertops, a walk-in pantry, and a large island. The master suite offers private backyard access and an ensuite bathroom with a soaking tub, dual vanities, and a walk-in shower.\nNeighborhood Description: Cedar Grove is known for its lush greenbelts, quiet walking trails, and friendly community events. It's an ideal location for families and retirees alike, with local parks, dog trails, and charming coffee shops that promote a laid-back lifestyle.", metadata={'row': 3, 'source': './final_real_estate_listing.c

# Test: Use a basic query and RAG approach to get LLM output for user query. 

In [20]:

query = """
Based on the houses listing in the context, suggest houses which are at least 3 bedrooms and 2 baths and are in the neighbourhood with lots of trees. 
A balance between suburban tranquility and access to urban amenities like restaurants and theaters. 
Try to provide the description as close to the context and in similar format. If no such listing of house is found, say nothing matching found.
"""

# No result test case
'''
query = """
Based on the houses listing in the context, suggest houses which are at least 6 bedrooms and 4 baths and are in the neighbourhood with lots of trees. 
A balance between suburban tranquility and access to urban amenities like restaurants and theaters. 
Try to provide the description as close to the context and in similar format. If no such listing of house is found, say nothing matching found.
"""
'''

use_chain_helper = True
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorStore.as_retriever())
    print(rag.run(query))
else:
    similar_docs = vectorStore.similarity_search(query=query, k=5)
    print(similar_docs[0])
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, query = query))


Neighborhood: Cedar Grove
Price: $720,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,100 sqft
Description: This beautifully updated single-story home in Cedar Grove blends modern comfort with natural serenity. Boasting 3 bedrooms and 2 full bathrooms, the home welcomes you with its airy open layout, vaulted ceilings, and large windows that frame picturesque views of the surrounding trees. The kitchen is designed for culinary enthusiasts, featuring quartz countertops, a walk-in pantry, and a large island. The master suite offers private backyard access and an ensuite bathroom with a soaking tub, dual vanities, and a walk-in shower.
Neighborhood Description: Cedar Grove is known for its lush greenbelts, quiet walking trails, and friendly community events. It's an ideal location for families and retirees alike, with local parks, dog trails, and charming coffee shops that promote a laid-back lifestyle.


# Step 4: Building the User Preference Interface

### Collect buyer preferences, such as the number of bedrooms, bathrooms, location, and other specific requirements from a set of questions or telling the buyer to enter their preferences in natural language.

In [9]:
# Sample question answer hardcoded

questions = [
    "How big do you want your house to be?", 
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?", 
    "How urban do you want your neighborhood to be?"
]

answers = [ 
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

### Buyer Preference Parsing: Implement logic to interpret and structure these preferences for querying the vector database.

In [63]:
house_plain_query = "Looking for a house as descirbed here. "
for i in range(len(answers)):
    house_plain_query += f" {answers[i]} "

print(house_plain_query)

Looking for a house as descirbed here.  A comfortable three-bedroom house with a spacious kitchen and a cozy living room.  A quiet neighborhood, good local schools, and convenient shopping options.  A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.  Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.  A balance between suburban tranquility and access to urban amenities like restaurants and theaters. 


In [70]:
question_and_answer = ""
for i in range(len(questions)):
    question_and_answer += f"\nQuestion: {questions[i]}"
    question_and_answer += f"\nAnswer: {answers[i]}"

print(question_and_answer)


Question: How big do you want your house to be?
Answer: A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
Question: What are 3 most important things for you in choosing this property?
Answer: A quiet neighborhood, good local schools, and convenient shopping options.
Question: Which amenities would you like?
Answer: A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
Question: Which transportation options are important to you?
Answer: Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.
Question: How urban do you want your neighborhood to be?
Answer: A balance between suburban tranquility and access to urban amenities like restaurants and theaters.


# Step 5: Searching Based on Preferences

### Semantic Search Implementation: Use the structured buyer preferences to perform a semantic search on the vector database, retrieving listings that most closely match the user's requirements.

In [60]:
def get_matching_listings(query) :

    # find top 3 semantically similar documents to the query
    result = vectorStore.similarity_search(query=buyer_llm_query, k=3)
    return result

In [64]:
buyer_query = house_plain_query
output = get_matching_listings(buyer_query)
print(output)

[Document(page_content='Neighborhood: Brookline\nPrice: $1,775,000\nBedrooms: 4\nBathrooms: 3\nHouse Size: 2,900 sqft\nDescription: This elegant 4-bedroom, 3-bathroom colonial in Brookline combines historical details with modern luxury. Step through the grand foyer into spacious living and dining rooms, adorned with crown molding and large bay windows. The chef’s kitchen opens up to a sunny breakfast nook and family room. Upstairs, the primary suite features a walk-in closet and spa bath. A fenced-in backyard and two-car garage round out this ideal suburban retreat.\nNeighborhood Description: Brookline offers a suburban feel with urban conveniences. Known for its excellent schools, leafy streets, and community-focused atmosphere, Brookline has easy access to downtown Boston via the Green Line, making it popular with families and professionals alike.', metadata={'row': 10, 'source': './final_real_estate_listing.csv'}), Document(page_content='Neighborhood: Green Oaks\nPrice: $800,000\nBe

### Listing Retrieval Logic: Fine-tune the retrieval algorithm to ensure that the most relevant listings are selected based on the semantic closeness to the buyer’s preferences.

#### TODO Improvement: Use Pydantic output parser for structured response rather than using example in the prompt

In [62]:

qa_query = """
Use the conversation below for my preferences for the house I am searching for. 

Conversation:
{question_and_answer}

Return details in structured format strictly as per the response example below. 
Extract bedrooms, bathrooms number and house size. Structure all other information as a summary in a description field. 
Do not paraphrase the answers from the user and ensure that the summary matches as close to the answers.
If some data is not specified for a field, keep it empty.

Response example: 
{response_format}

"""

prompt = PromptTemplate(
    template=qa_query,
    input_variables=["question_and_answer", "response_format"],
    #partial_variables={"format_instructions": parser.get_format_instructions},
)

response_format = """
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft
Description: I am looking for a charming 3-bedroom, 2-bathroom home with energy-efficient features such as solar panels and a well-insulated structure. Natural light should floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area with a spacious backyard. I embrace sustainable living without compromising on style.
"""

llm_query = prompt.format(question_and_answer=question_and_answer, response_format=response_format)
print(llm_query)

In [71]:
llm_output = llm.invoke(llm_query)
print(llm_output)

Bedrooms: 3
Bathrooms: 
House Size: 
Description: I am searching for a comfortable three-bedroom house with a spacious kitchen and a cozy living room. My top priorities are a quiet neighborhood, good local schools, and convenient shopping options. I would also like a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. In terms of transportation, I am looking for easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. I prefer a neighborhood that strikes a balance between suburban tranquility and access to urban amenities like restaurants and theaters.


In [72]:
buyer_llm_query = llm_output
output = get_matching_listings(buyer_llm_query)
print(output)

[Document(page_content='Neighborhood: Brookline\nPrice: $1,775,000\nBedrooms: 4\nBathrooms: 3\nHouse Size: 2,900 sqft\nDescription: This elegant 4-bedroom, 3-bathroom colonial in Brookline combines historical details with modern luxury. Step through the grand foyer into spacious living and dining rooms, adorned with crown molding and large bay windows. The chef’s kitchen opens up to a sunny breakfast nook and family room. Upstairs, the primary suite features a walk-in closet and spa bath. A fenced-in backyard and two-car garage round out this ideal suburban retreat.\nNeighborhood Description: Brookline offers a suburban feel with urban conveniences. Known for its excellent schools, leafy streets, and community-focused atmosphere, Brookline has easy access to downtown Boston via the Green Line, making it popular with families and professionals alike.', metadata={'row': 10, 'source': './final_real_estate_listing.csv'}), Document(page_content='Neighborhood: Cambridge\nPrice: $1,450,000\nB

# Step 6: Personalizing Listing Descriptions

### LLM Augmentation
For each retrieved listing, use the LLM to augment the description, tailoring it to resonate with the buyer’s specific preferences. 
This involves subtly emphasizing aspects of the property that align with what the buyer is looking for.

#### Maintaining Factual Integrity: Ensure that the augmentation process enhances the appeal of the listing without altering factual information.

In [82]:
preference = """
Bedrooms: 3
Bathrooms: 
House Size: 
Description: I am searching for a comfortable three-bedroom house with a spacious kitchen and a cozy living room. My top priorities are a quiet neighborhood, good local schools, and convenient shopping options. I would also like a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. In terms of transportation, I am looking for easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. I prefer a neighborhood that strikes a balance between suburban tranquility and access to urban amenities like restaurants and theaters.
"""

In [83]:
query = """
You are a real estate agent. 
Based on the houses listings in the context, suggest houses which match the user preference mentioned below.
Return the description summary for each matching house emphasizing how it is matching with the user provided preference.
Do not add any new factual infromation in your summary. Use the information from the context only and use them to create sales pitch around how they match the user preference.

If no such listing of house is found, say nothing matching found.
"""

In [84]:
similar_listings = get_matching_listings(preference)
#print(similar_listings[0])
prompt = PromptTemplate(
    template="{query}\nUser Preference: {user_preference}\nContext: {context}",
    input_variables=["query", "user_preference", "context"],
)

chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
print(chain.run(input_documents=similar_listings, query = query, user_preference=preference))

 The neighborhood also offers easy access to major highways and public transportation options.

Matching houses found:

1. Neighborhood: Brookline
Price: $1,775,000
Bedrooms: 6
Bathrooms: 3
House Size: 2,900 sqft
Description: This stunning 6-bedroom colonial in Brookline offers the perfect combination of suburban tranquility and urban convenience. With a spacious kitchen, cozy living room, and a fenced-in backyard for gardening, this home has everything you're looking for. The quiet neighborhood is known for its excellent schools and easy access to downtown Boston via the Green Line. Don't miss out on this ideal family home.

2. Neighborhood: Cambridge
Price: $1,450,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,100 sqft
Description: Located just minutes from Harvard and MIT, this sophisticated 3-bedroom townhouse in Cambridge is the perfect blend of classic charm and modern updates. With a renovated kitchen, open-concept living area, and private patio, this home is ideal for entertaining.